In [ ]:
import numpy as np
from scipy.linalg import eigh
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import robust_laplacian
from Mesh import Mesh
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.sparse import csr_matrix, dia_matrix, csc_matrix
from scipy import sparse
from tqdm import trange

In [ ]:
# ============ DEVICE SETUP ============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ============ NETWORK ARCHITECTURE ============
class Sin(nn.Module):
    """Sine activation function"""
    def forward(self, x):
        return torch.sin(x)


class EigenfunctionNN(nn.Module):
    """
    Neural network to learn eigenfunctions on point clouds.
    Input: 3D coordinates (x, y, z)
    Output: eigenfunction value u(x,y,z) and eigenvalue λ
    """
    def __init__(self, hidden_dim=64, input_dim=3, initial_eigenvalue=0.0):
        super().__init__()
        self.activation = Sin()
        
        # Learnable eigenvalue with better initialization
        self.eigenvalue_layer = nn.Linear(1, 1, bias=False)
        with torch.no_grad():
            self.eigenvalue_layer.weight.fill_(initial_eigenvalue)
        
        # Network layers - concatenate eigenvalue at each layer
        self.fc1 = nn.Linear(input_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc2 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc3 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc4 = nn.Linear(hidden_dim + 1, 1)  # +1 for eigenvalue
        
    def forward(self, x):
        """
        Args:
            x: (N, 3) point cloud coordinates
        Returns:
            u: (N, 1) eigenfunction values
            eigenvalue: scalar learnable eigenvalue
        """
        # Learn eigenvalue and broadcast to match batch size
        eigenvalue = torch.abs(self.eigenvalue_layer(torch.ones(1, 1).to(x.device)))
        eigenvalue_expanded = eigenvalue.expand(x.shape[0], 1)  # (N, 1)
        
        # Forward pass - concatenate eigenvalue at each layer
        h = torch.cat([x, eigenvalue_expanded], dim=1)  # (N, input_dim+1)
        h = self.activation(self.fc1(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc2(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc3(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        u = self.fc4(h)
        
        return u, eigenvalue


# ============ LOSS COMPUTATION ============
def compute_eigenvalue_loss(u, eigenvalue, L, M, X, device):
    """
    Compute residual for Lu = λMu using discrete operators.
    
    Args:
        u: (N, 1) predicted eigenfunction values
        eigenvalue: scalar predicted eigenvalue
        L: (N, N) Laplacian matrix (scipy sparse: csr, csc, dia, etc.)
        M: (N, N) Mass matrix (scipy sparse: csr, csc, dia, etc.)
        X: (N, 3) point cloud coordinates
        device: torch device
    
    Returns:
        loss: MSE of residual ||Lu - λMu||²
    """
    u_flat = u.squeeze()  # (N,)
    
    # Convert sparse matrices to torch sparse tensors if needed
    if sparse.issparse(L):
        L_torch = sparse_to_torch(L, device)
        M_torch = sparse_to_torch(M, device)
    else:
        L_torch = L
        M_torch = M
    
    # Compute Lu and λMu
    Lu = torch.sparse.mm(L_torch, u_flat.unsqueeze(1)).squeeze()
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    lMu = eigenvalue * Mu
    
    # Residual loss
    residual = Lu - lMu
    loss = torch.mean(residual ** 2)
    
    return loss, Lu, Mu


def compute_normalization_loss(u, M, device):
    """
    Enforce u^T M u = 1 (mass-matrix normalization).
    """
    u_flat = u.squeeze()
    
    if sparse.issparse(M):
        M_torch = sparse_to_torch(M, device)
    else:
        M_torch = M
    
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    norm_squared = torch.dot(u_flat, Mu)
    
    # Penalize deviation from unit norm
    loss = (norm_squared - 1.0) ** 2
    
    return loss


def compute_orthogonality_loss(u, previous_eigenfunctions, M, device):
    """
    Enforce u ⊥ u_i for all previously found eigenfunctions.
    Uses M-orthogonality: u^T M u_i = 0
    """
    if len(previous_eigenfunctions) == 0:
        return torch.tensor(0.0, device=device)
    
    u_flat = u.squeeze()
    ortho_loss = torch.tensor(0.0, device=device)
    
    if sparse.issparse(M):
        M_torch = sparse_to_torch(M, device)
    else:
        M_torch = M
    
    for u_prev in previous_eigenfunctions:
        u_prev_flat = u_prev.squeeze()
        # Compute u^T M u_prev
        Mu_prev = torch.sparse.mm(M_torch, u_prev_flat.unsqueeze(1)).squeeze()
        overlap = torch.dot(u_flat, Mu_prev)
        ortho_loss += overlap ** 2
    
    return ortho_loss


# ============ UTILITY FUNCTIONS ============
def sparse_to_torch(sparse_matrix, device):
    """Convert scipy sparse matrix to torch sparse tensor."""
    # Handle any scipy sparse format
    if sparse.issparse(sparse_matrix):
        coo = sparse_matrix.tocoo()
    else:
        raise ValueError(f"Expected scipy sparse matrix, got {type(sparse_matrix)}")
    
    indices = torch.LongTensor(np.vstack((coo.row, coo.col)))
    values = torch.FloatTensor(coo.data)
    shape = coo.shape
    return torch.sparse_coo_tensor(indices, values, shape).to(device)


def initialize_weights(m):
    """Reinitialize network weights for finding next eigenfunction."""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


# ============ MAIN TRAINING FUNCTION ============
def train_eigenvalue_pinn(X, L, M, hidden_dim=64, epochs=20000, 
                          lr=1e-3, num_eigenfunctions=5, 
                          convergence_threshold=1e-7,
                          ortho_weight=1.0):
    """print(f"The true Eigenvalues are: {np.array2string(eigvals[:10], formatter={'float': lambda x: f'{x:.3f}'})}")
print(f"The predicted Eigenvalues are: {np.array2string(np.array(eigenvalues), formatter={'float': lambda x: f'{x:.3f}'})}")
    Train PINN to solve Lu = λMu eigenvalue problem.
    
    Args:
        X: (N, 3) point cloud coordinates
        L: (N, N) Laplacian matrix (scipy sparse: csr, csc, dia, etc.)
        M: (N, N) Mass matrix (scipy sparse: csr, csc, dia, etc.)
        hidden_dim: Hidden layer dimension
        epochs: Training epochs per eigenfunction
        lr: Learning rate
        num_eigenfunctions: Number of eigenfunctions to find
        convergence_threshold: Threshold for detecting convergence
        ortho_weight: Weight for orthogonality loss (increase if eigenfunctions overlap)
    
    Returns:
        eigenvalues: List of found eigenvalues
        eigenfunctions: List of (N, 1) eigenfunctions
        loss_history: Training loss history
    """
    
    # Convert inputs to torch (handle both numpy arrays and torch tensors)
    if isinstance(X, np.ndarray):
        X_torch = torch.FloatTensor(X).to(device)
    elif isinstance(X, torch.Tensor):
        X_torch = X.float().to(device)  # Ensure float32
    else:
        raise ValueError(f"X must be numpy array or torch tensor, got {type(X)}")
    
    X_torch.requires_grad = True
    
    # Storage for results
    eigenvalues = []
    eigenfunctions = []
    all_models = []
    loss_history = {'total': [], 'eigenvalue': [], 'normalization': [], 'orthogonality': []}
    
    print(f"Training on device: {device}")
    print(f"Point cloud size: {X.shape[0]} points")
    print(f"Matrix format: L is {type(L).__name__}, M is {type(M).__name__}")
    
    # ============ ITERATIVE EIGENFUNCTION DISCOVERY ============
    for eig_idx in range(num_eigenfunctions):
        print(f"\n{'='*60}")
        print(f"Finding eigenfunction {eig_idx + 1}/{num_eigenfunctions}")
        print(f"{'='*60}")
        
        # Initialize network with progressively larger eigenvalue guess
        # For Laplacian: smallest eigenvalue is 0
        if eig_idx == 0:
            initial_eigenvalue = 0.0  # First eigenvalue for Laplacian
        elif eig_idx > 0:
            # Use previous eigenvalue as lower bound + small increment
            initial_eigenvalue = eigenvalues[-1] + 0.15
        else:
            initial_eigenvalue = eig_idx * 0.2
        
        model = EigenfunctionNN(hidden_dim=hidden_dim, input_dim=X.shape[1], 
                               initial_eigenvalue=initial_eigenvalue).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.999, 0.9999))
        
        best_model = None
        best_loss = float('inf')
        loss_slope_history = []
        
        for epoch in range(epochs):
            optimizer.zero_grad()
            
            # Forward pass
            u, eigenvalue = model(X_torch)
            
            # Compute losses
            eig_loss, Lu, Mu = compute_eigenvalue_loss(u, eigenvalue, L, M, X_torch, device)
            norm_loss = compute_normalization_loss(u, M, device)
            ortho_loss = compute_orthogonality_loss(u, eigenfunctions, M, device)
            
            # Total loss with weighting
            total_loss = eig_loss + norm_loss + ortho_weight * ortho_loss
            
            # Backward pass
            total_loss.backward()
            optimizer.step()
            
            # Track loss slope for convergence detection
            loss_slope_history.append(eig_loss.item())
            if len(loss_slope_history) > 1000:
                loss_slope_history.pop(0)
                slope = np.mean(np.diff(loss_slope_history))
            else:
                slope = 0.0
            
            # Save best model
            if eig_loss.item() < best_loss:
                best_loss = eig_loss.item()
                best_model = copy.deepcopy(model)
            
            # Logging
            if epoch % 500 == 0:
                print(f"Epoch {epoch:5d} | λ={eigenvalue.item():.6f} | "
                      f"Eig loss={eig_loss.item():.2e} | Norm loss={norm_loss.item():.2e} | "
                      f"Ortho loss={ortho_loss.item():.2e} | Slope={slope:.2e}")
            
            # Store history
            loss_history['total'].append(total_loss.item())
            loss_history['eigenvalue'].append(eig_loss.item())
            loss_history['normalization'].append(norm_loss.item())
            loss_history['orthogonality'].append(ortho_loss.item())
            
            # Check for convergence and reinitialize if stuck
            if epoch > 5000 and len(loss_slope_history) == 1000:
                if abs(slope) < convergence_threshold:
                    print(f"Converged at epoch {epoch}!")
                    break
        
        # Store results for this eigenfunction
        with torch.no_grad():
            u_final, eigenvalue_final = best_model(X_torch)
            eigenvalues.append(eigenvalue_final.item())
            eigenfunctions.append(u_final.detach())
            all_models.append(best_model)
        
        print(f"\nFound eigenvalue: λ_{eig_idx} = {eigenvalue_final.item():.6f}")
    
    return eigenvalues, eigenfunctions, all_models, loss_history


In [ ]:
"""

THIS ONE ACTUALLY WORKS OK

"""



# ============ DEVICE SETUP ============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ============ NETWORK ARCHITECTURE ============
class Sin(nn.Module):
    """Sine activation function"""
    def forward(self, x):
        return torch.sin(x)


class EigenfunctionNN(nn.Module):
    """
    Neural network to learn eigenfunctions on point clouds.
    Input: 3D coordinates (x, y, z)
    Output: eigenfunction value u(x,y,z) and eigenvalue λ
    """
    def __init__(self, hidden_dim=64, input_dim=3, initial_eigenvalue=0.0):
        super().__init__()
        self.activation = Sin()
        
        # Learnable eigenvalue with better initialization
        self.eigenvalue_layer = nn.Linear(1, 1, bias=False)
        with torch.no_grad():
            self.eigenvalue_layer.weight.fill_(initial_eigenvalue)
        
        # Network layers - concatenate eigenvalue at each layer
        self.fc1 = nn.Linear(input_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc2 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc3 = nn.Linear(hidden_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc4 = nn.Linear(hidden_dim + 1, 1)  # +1 for eigenvalue
        
    def forward(self, x):
        """
        Args:
            x: (N, 3) point cloud coordinates
        Returns:
            u: (N, 1) eigenfunction values
            eigenvalue: scalar learnable eigenvalue
        """
        # Learn eigenvalue and broadcast to match batch size
        eigenvalue = torch.abs(self.eigenvalue_layer(torch.ones(1, 1).to(x.device)))
        eigenvalue_expanded = eigenvalue.expand(x.shape[0], 1)  # (N, 1)
        
        # Forward pass - concatenate eigenvalue at each layer
        h = torch.cat([x, eigenvalue_expanded], dim=1)  # (N, input_dim+1)
        h = self.activation(self.fc1(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc2(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        h = self.activation(self.fc3(h))
        
        h = torch.cat([h, eigenvalue_expanded], dim=1)  # (N, hidden_dim+1)
        u = self.fc4(h)
        
        return u, eigenvalue


# ============ LOSS COMPUTATION ============
def compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch):
    """
    Compute residual for Lu = λMu using discrete operators.
    Returns:
        loss: MSE of residual ||Lu - λMu||²
    """
    u_flat = u.squeeze()  # (N,)
    
    # Compute Lu and λMu
    Lu = torch.sparse.mm(L_torch, u_flat.unsqueeze(1)).squeeze()
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    residual = Lu - eigenvalue * Mu
    
    return torch.mean(residual ** 2), Lu, Mu


def compute_normalization_loss(u, M_torch):
    """
    Enforce u^T M u = 1 (mass-matrix normalization).
    """
    u_flat = u.squeeze()    
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    norm_squared = torch.dot(u_flat, Mu)

    return (norm_squared - 1.0) ** 2


def compute_orthogonality_loss(u, previous_eigenfunctions, M_torch):
    """
    Enforce u ⊥ u_i for all previously found eigenfunctions.
    Uses M-orthogonality: u^T M u_i = 0
    """
    if not previous_eigenfunctions:
        return torch.tensor(0.0, device=M_torch.device)
    
    u_flat = u.squeeze()
    ortho_loss = torch.tensor(0.0, device=M_torch.device)
    
    
    for u_prev in previous_eigenfunctions:
        u_prev_flat = u_prev.squeeze()
        # Compute u^T M u_prev
        Mu_prev = torch.sparse.mm(M_torch, u_prev_flat.unsqueeze(1)).squeeze()
        overlap = torch.dot(u_flat, Mu_prev)
        ortho_loss += overlap ** 2
    
    return ortho_loss


# ============ UTILITY FUNCTIONS ============
def sparse_to_torch(sparse_matrix, device):
    """Convert scipy sparse matrix to torch sparse tensor."""
    # Handle any scipy sparse format
    if sparse.issparse(sparse_matrix):
        coo = sparse_matrix.tocoo()
    else:
        raise ValueError(f"Expected scipy sparse matrix, got {type(sparse_matrix)}")
    
    indices = torch.LongTensor(np.vstack((coo.row, coo.col)))
    values = torch.FloatTensor(coo.data)
    shape = coo2,.shape
    return torch.sparse_coo_tensor(indices, values, shape).to(device)


def initialize_weights(m):
    """Reinitialize network weights for finding next eigenfunction."""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


# ============ MAIN TRAINING FUNCTION ============
def train_eigenvalue_pinn(X, L, M, hidden_dim=64, epochs=20000, 
                          lr=1e-3, num_eigenfunctions=5, 
                          convergence_threshold=1e-7,
                          ortho_weight=1.0):
    """print(f"The true Eigenvalues are: {np.array2string(eigvals[:10], formatter={'float': lambda x: f'{x:.3f}'})}")
print(f"The predicted Eigenvalues are: {np.array2string(np.array(eigenvalues), formatter={'float': lambda x: f'{x:.3f}'})}")
    Train PINN to solve Lu = λMu eigenvalue problem.
    
    Args:
        X: (N, 3) point cloud coordinates
        L: (N, N) Laplacian matrix (scipy sparse: csr, csc, dia, etc.)
        M: (N, N) Mass matrix (scipy sparse: csr, csc, dia, etc.)
        hidden_dim: Hidden layer dimension
        epochs: Training epochs per eigenfunction
        lr: Learning rate
        num_eigenfunctions: Number of eigenfunctions to find
        convergence_threshold: Threshold for detecting convergence
        ortho_weight: Weight for orthogonality loss (increase if eigenfunctions overlap)
    
    Returns:
        eigenvalues: List of found eigenvalues
        eigenfunctions: List of (N, 1) eigenfunctions
        loss_history: Training loss history
    """
    
    # Prepare inputs
    X_torch = torch.as_tensor(X, dtype=torch.float32, device=device)
    X_torch.requires_grad = True

    # Pre-convert sparse matrices
    L_torch = sparse_to_torch(L, device) if sparse.issparse(L) else L.to(device)
    M_torch = sparse_to_torch(M, device) if sparse.issparse(M) else M.to(device)
    
    # Storage for results
    eigenvalues = []
    eigenfunctions = []
    all_models = []
    loss_history = {'total': [], 'eig': [], 'norm': [], 'ortho': []}
    
    print(f"Training on device: {device}")
    print(f"Point cloud size: {X.shape[0]} points")
    print(f"Matrix format: L is {type(L).__name__}, M is {type(M).__name__}")
    
    # ============ ITERATIVE EIGENFUNCTION DISCOVERY ============
    for eig_idx in range(num_eigenfunctions):
        print(f"\n{'='*60}")
        print(f"Finding eigenfunction {eig_idx + 1}/{num_eigenfunctions}")
        print(f"{'='*60}")
        
        # Initialize network with progressively larger eigenvalue guess
        # For Laplacian: smallest eigenvalue is 0
        if eig_idx == 0:
            initial_eigenvalue = 0.0  # First eigenvalue for Laplacian
        elif eig_idx > 0:
            # Use previous eigenvalue as lower bound + small increment
            initial_eigenvalue = eigenvalues[-1] + 0.15
        else:
            initial_eigenvalue = eig_idx * 0.2
        
        model = EigenfunctionNN(hidden_dim=hidden_dim, input_dim=X.shape[1], 
                               initial_eigenvalue=initial_eigenvalue).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.999, 0.9999))
        
        best_model = None
        best_loss = float('inf')
        ema_slope = 1.0
        prev_loss = None
        
        for epoch in trange(epochs, desc=f"Eigen {eig_idx+1}/{num_eigenfunctions}"):
            optimizer.zero_grad()
            
            # Forward pass
            u, eigenvalue = model(X_torch)
            
            # Compute losses
            eig_loss, _, _ = compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch)
            norm_loss = compute_normalization_loss(u, M_torch)
            ortho_loss = compute_orthogonality_loss(u, eigenfunctions, M_torch)
            
            # Total loss with weighting
            total_loss = eig_loss + norm_loss + ortho_weight * ortho_loss
            
            # Backward pass
            total_loss.backward()
            optimizer.step()
            
            # Convergence tracking
            if prev_loss is not None:
                ema_slope = 0.75 * ema_slope + 0.25 * abs(prev_loss - eig_loss.item())
            prev_loss = eig_loss.item()

            if ema_slope < convergence_threshold and epoch > 2000:
                print(f"Converged at epoch {epoch}")
                break

            # Save best
            if eig_loss.item() < best_loss:
                best_loss = eig_loss.item()
                best_model = copy.deepcopy(model)

            # Logging every 500 epochs
            if epoch % 500 == 0:
                print(f"Epoch {epoch:5d} | λ={eigenvalue.item():.6f} | "
                      f"Eig={eig_loss.item():.2e} | Norm={norm_loss.item():.2e} | "
                      f"Ortho={ortho_loss.item():.2e} | EMA slope={ema_slope:.2e}")

            # Lightweight history
            if epoch % 100 == 0:
                loss_history['total'].append(total_loss.item())
                loss_history['eig'].append(eig_loss.item())
                loss_history['norm'].append(norm_loss.item())
                loss_history['ortho'].append(ortho_loss.item())

        # Store results
        with torch.no_grad():
            u_final, eigenvalue_final = best_model(X_torch)
            eigenvalues.append(eigenvalue_final.item())
            eigenfunctions.append(u_final.detach())
            all_models.append(best_model)

        print(f"\nFound eigenvalue: λ_{eig_idx} = {eigenvalue_final.item():.6f}")
    
    return eigenvalues, eigenfunctions, all_models, loss_history


In [125]:
 # ============ DEVICE SETUP ============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ============ NETWORK ARCHITECTURE ============
class Sin(nn.Module):
    """Sine activation function"""
    def forward(self, x):
        return torch.sin(x)


class EigenfunctionNN(nn.Module):
    """
    Neural network to learn eigenfunctions on point clouds.
    Input: 3D coordinates (x, y, z)
    Output: eigenfunction value u(x,y,z) and eigenvalue λ
    """
    def __init__(self, hidden_dim=64, input_dim=3, initial_eigenvalue=0.0):
        super().__init__()
        self.activation = Sin()
        
        # Learnable eigenvalue with better initialization
        self.eigenvalue_layer = nn.Linear(1, 1, bias=False)
        with torch.no_grad():
            self.eigenvalue_layer.weight.fill_(initial_eigenvalue)
        
        # Network layers - ONLY concatenate eigenvalue at the first layer
        # This keeps the eigenvalue informative but prevents over-coupling in deeper layers
        self.fc1 = nn.Linear(input_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        """
        Args:
            x: (N, 3) point cloud coordinates
        Returns:
            u: (N, 1) eigenfunction values
            eigenvalue: scalar learnable eigenvalue
        """
        # Learn eigenvalue and broadcast to match batch size
        # Use torch.clamp(..., min=0.0) to enforce non-negativity
        eigenvalue = torch.clamp(self.eigenvalue_layer(torch.ones(1, 1).to(x.device)), min=0.0)
        eigenvalue_expanded = eigenvalue.expand(x.shape[0], 1)  # (N, 1)
        
        # Forward pass - concatenate eigenvalue ONLY at the input
        h = torch.cat([x, eigenvalue_expanded], dim=1)  # (N, input_dim+1)
        h = self.activation(self.fc1(h))
        
        # No eigenvalue concatenation in hidden layers
        h = self.activation(self.fc2(h))
        h = self.activation(self.fc3(h))
        
        u = self.fc4(h)
        
        return u, eigenvalue


# ============ LOSS COMPUTATION ============
def compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch):
    """
    Compute residual for Lu = λMu using discrete operators.
    Returns:
        loss: MSE of residual ||Lu - λMu||²
    """
    u_flat = u.squeeze()  # (N,)
    
    # Compute Lu and λMu
    # Ensure u_flat is (N, 1) for sparse.mm, which expects a 2D tensor
    Lu = torch.sparse.mm(L_torch, u_flat.unsqueeze(1)).squeeze()
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    residual = Lu - eigenvalue * Mu
    
    return torch.mean(residual ** 2), Lu, Mu


def compute_normalization_loss(u, M_torch):
    """
    Enforce u^T M u = 1 (mass-matrix normalization).
    """
    u_flat = u.squeeze()    
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    norm_squared = torch.dot(u_flat, Mu)

    return (norm_squared - 1.0) ** 2


def compute_orthogonality_loss(u, previous_eigenfunctions, M_torch):
    """
    Enforce u ⊥ u_i for all previously found eigenfunctions.
    Uses M-orthogonality: u^T M u_i = 0
    """
    if not previous_eigenfunctions:
        return torch.tensor(0.0, device=M_torch.device)
    
    u_flat = u.squeeze()
    ortho_loss = torch.tensor(0.0, device=M_torch.device)
    
    
    for u_prev in previous_eigenfunctions:
        u_prev_flat = u_prev.squeeze()
        # Compute u^T M u_prev
        Mu_prev = torch.sparse.mm(M_torch, u_prev_flat.unsqueeze(1)).squeeze()
        overlap = torch.dot(u_flat, Mu_prev)
        ortho_loss += overlap ** 2
    
    return ortho_loss


def compute_ordering_loss(eigenvalue, previous_eigenvalue, margin=1e-3):
    """
    Enforce λ_i > λ_{i-1} + margin.
    This is crucial for preventing mode collapse to a lower-energy mode.
    """
    if previous_eigenvalue is None:
        return torch.tensor(0.0, device=eigenvalue.device)
    
    # max(0, λ_{i-1} + margin - λ_i)^2
    loss = torch.max(
        torch.tensor(0.0, device=eigenvalue.device), 
        previous_eigenvalue.detach() + margin - eigenvalue
    ) ** 2
    return loss.squeeze()


# ============ UTILITY FUNCTIONS ============
def sparse_to_torch(sparse_matrix, device):
    """Convert scipy sparse matrix to torch sparse tensor."""
    # Handle any scipy sparse format
    if sparse.issparse(sparse_matrix):
        coo = sparse_matrix.tocoo()
    else:
        raise ValueError(f"Expected scipy sparse matrix, got {type(sparse_matrix)}")
    
    indices = torch.LongTensor(np.vstack((coo.row, coo.col)))
    values = torch.FloatTensor(coo.data)
    shape = coo.shape
    return torch.sparse_coo_tensor(indices, values, shape).to(device)


def initialize_weights(m):
    """Reinitialize network weights for finding next eigenfunction."""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


# ============ MAIN TRAINING FUNCTION ============
def train_eigenvalue_pinn(X, L, M, hidden_dim=64, epochs=20000, 
                          lr=1e-3, num_eigenfunctions=10, 
                          convergence_threshold=1e-7,
                          ortho_weight=100.0,  # CRITICAL INCREASE: From 1.0 to 100.0
                          order_weight=100.0,   # NEW: Weight for the ordering constraint
                          initial_lambda_step=0.3): # CRITICAL INCREASE: From 0.15 to 0.3
    """
    Train PINN to solve Lu = λMu eigenvalue problem.
    """
    
    # Prepare inputs
    X_torch = torch.as_tensor(X, dtype=torch.float32, device=device)
    X_torch.requires_grad = True

    # Pre-convert sparse matrices
    L_torch = sparse_to_torch(L, device) if sparse.issparse(L) else L.to(device)
    M_torch = sparse_to_torch(M, device) if sparse.issparse(M) else M.to(device)
    
    # Storage for results
    eigenvalues = []
    eigenfunctions = []
    all_models = []
    loss_history = {'total': [], 'eig': [], 'norm': [], 'ortho': [], 'order': []}
    
    print(f"Training on device: {device}")
    print(f"Point cloud size: {X.shape[0]} points")
    print(f"Matrix format: L is {type(L).__name__}, M is {type(M).__name__}")
    
    # ============ ITERATIVE EIGENFUNCTION DISCOVERY ============
    for eig_idx in range(num_eigenfunctions):
        print(f"\n{'='*60}")
        print(f"Finding eigenfunction {eig_idx + 1}/{num_eigenfunctions}")
        print(f"Hyperparams: λ_step={initial_lambda_step:.2f}, ortho_w={ortho_weight:.1f}, order_w={order_weight:.1f}")
        print(f"{'='*60}")
        
        # Determine previous eigenvalue and initial guess
        previous_eigenvalue = None
        if eig_idx == 0:
            initial_eigenvalue = 0.0
        else:
            previous_eigenvalue = torch.tensor(eigenvalues[-1], dtype=torch.float32, device=device)
            # Use larger, non-fixed step size
            initial_eigenvalue = eigenvalues[-1] + initial_lambda_step 
            
        
        # Initialize network
        model = EigenfunctionNN(hidden_dim=hidden_dim, input_dim=X.shape[1], 
                               initial_eigenvalue=initial_eigenvalue).to(device)
        
        # REINITIALIZE EVERYTHING FOR NOW (can be optimized later)
        model.apply(initialize_weights) 
        
        # Re-set initial eigenvalue guess, as initialize_weights overwrites it
        if previous_eigenvalue is not None:
             with torch.no_grad():
                model.eigenvalue_layer.weight.fill_(initial_eigenvalue)


        optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.999, 0.9999))
        
        # --- Convergence Tracking Setup ---
        best_model = None
        best_eig_loss = float('inf')
        ema_slope = 1.0
        prev_loss = None
        
        # --- Training Loop ---
        for epoch in trange(epochs, desc=f"Eigen {eig_idx+1}/{num_eigenfunctions}"):
            optimizer.zero_grad()
            
            # Forward pass
            u, eigenvalue = model(X_torch)
            
            # Compute losses
            eig_loss, _, _ = compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch)
            norm_loss = compute_normalization_loss(u, M_torch)
            ortho_loss = compute_orthogonality_loss(u, eigenfunctions, M_torch)
            order_loss = compute_ordering_loss(eigenvalue, previous_eigenvalue) # NEW LOSS
            
            # Total loss with weighting
            total_loss = eig_loss + norm_loss + \
                         ortho_weight * ortho_loss + \
                         order_weight * order_loss # APPLY WEIGHT
            
            # Backward pass
            total_loss.backward()
            optimizer.step()
            
            # Convergence tracking (Use Eig loss for convergence)
            if prev_loss is not None:
                ema_slope = 0.75 * ema_slope + 0.25 * abs(prev_loss - eig_loss.item())
            prev_loss = eig_loss.item()

            # Wait longer for convergence check (e.g., after 5000 epochs)
            # And use a higher threshold (e.g., 1e-5) as convergence can be noisy
            if ema_slope < convergence_threshold * 10.0 and epoch > 5000:
                print(f"Converged at epoch {epoch}")
                break

            # Save best model based on Eig loss
            if eig_loss.item() < best_eig_loss:
                best_eig_loss = eig_loss.item()
                best_model = copy.deepcopy(model)

            # Logging every 500 epochs
            if epoch % 500 == 0:
                print(f"Epoch {epoch:5d} | λ={eigenvalue.item():.6f} | "
                      f"Eig={eig_loss.item():.2e} | Norm={norm_loss.item():.2e} | "
                      f"Ortho={ortho_loss.item():.2e} | Order={order_loss.item():.2e} | " # NEW LOG
                      f"EMA slope={ema_slope:.2e}")

            # Lightweight history
            if epoch % 100 == 0:
                loss_history['total'].append(total_loss.item())
                loss_history['eig'].append(eig_loss.item())
                loss_history['norm'].append(norm_loss.item())
                loss_history['ortho'].append(ortho_loss.item())
                loss_history['order'].append(order_loss.item())

        # Store results
        if best_model is None:
             print("Warning: No best model found (first epoch was best, using final model).")
             best_model = model
             
        with torch.no_grad():
            u_final, eigenvalue_final = best_model(X_torch)
            eigenvalues.append(eigenvalue_final.item())
            
            # Re-normalize the eigenfunction after training
            u_flat = u_final.squeeze()
            Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
            norm_squared = torch.dot(u_flat, Mu)
            u_normalized = u_final / torch.sqrt(norm_squared)

            eigenfunctions.append(u_normalized.detach())
            all_models.append(best_model)

        print(f"\nFound eigenvalue: λ_{eig_idx} = {eigenvalue_final.item():.6f}")
    
    return eigenvalues, eigenfunctions, all_models, loss_history

In [126]:
m = Mesh('bunny.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

X = (m.verts - centroid)/std_max

m = Mesh(verts = X, connectivity = m.connectivity)

L, M = robust_laplacian.point_cloud_laplacian(X)

print('Computing Laplacian')
K_igl, M_igl = m.computeLaplacian()

# following Finite Elements methodology 
# K is stiffness matrix, M is mass matrix
# The problem to solve becomes 
# K*u = lambda * M*u
print('Computing eigen values')
eigvals, eigvecs = eigh(K_igl,M_igl)

# send all relevant numpy arrays to torch tensors
K_ = torch.from_numpy(K_igl).float().to(device)
M_ = torch.from_numpy(M_igl).float().to(device)
X_ = torch.from_numpy(m.verts).float().to(device)



eigenvalues, eigenfunctions, models, history = train_eigenvalue_pinn(
    X_, K_, M_, 
    hidden_dim=128, 
    epochs=5000, 
    lr=1e-3, 
    num_eigenfunctions=10
)

print("\n" + "="*60)
print("RESULTS")
print("="*60)
for i, lam in enumerate(eigenvalues):
    print(f"λ_{i} = {lam:.6f}")

Computing Laplacian
Computing eigen values
Training on device: cuda
Point cloud size: 2503 points
Matrix format: L is Tensor, M is Tensor

Finding eigenfunction 1/10
Hyperparams: λ_step=0.30, ortho_w=100.0, order_w=100.0


Eigen 1/10:   1%|          | 30/5000 [00:00<00:16, 292.40it/s]

Epoch     0 | λ=0.000000 | Eig=1.77e-06 | Norm=8.32e-01 | Ortho=0.00e+00 | Order=0.00e+00 | EMA slope=1.00e+00


Eigen 1/10:  12%|█▏        | 590/5000 [00:01<00:09, 444.53it/s]

Epoch   500 | λ=0.000000 | Eig=9.06e-06 | Norm=3.35e-02 | Ortho=0.00e+00 | Order=0.00e+00 | EMA slope=2.43e-07


...

Epoch  4000 | λ=1.498295 | Eig=2.15e-07 | Norm=9.93e-01 | Ortho=2.58e-03 | Order=0.00e+00 | EMA slope=2.13e-09


Eigen 10/10:  91%|█████████ | 4534/5000 [00:17<00:01, 274.18it/s]

Epoch  4500 | λ=1.511043 | Eig=6.61e-08 | Norm=9.97e-01 | Ortho=1.59e-03 | Order=0.00e+00 | EMA slope=4.15e-10


Eigen 10/10: 100%|██████████| 5000/5000 [00:18<00:00, 266.28it/s]


Found eigenvalue: λ_9 = 1.508162

RESULTS
λ_0 = 0.000000
λ_1 = 0.320646
λ_2 = 0.581464
λ_3 = 0.786167
λ_4 = 0.849548
λ_5 = 0.901922
λ_6 = 1.235297
λ_7 = 1.333021
λ_8 = 1.361330
λ_9 = 1.508162


In [124]:
print(f"The true Eigenvalues are: {np.array2string(eigvals[:10], formatter={'float': lambda x: f'{x:.3f}'})}")
print(f"The pred Eigenvalues are: {np.array2string(np.array(eigenvalues), formatter={'float': lambda x: f'{x:.3f}'})}")

The true Eigenvalues are: [0.000 0.160 0.425 0.438 0.538 0.612 0.896 1.274 1.496 1.643]
The pred Eigenvalues are: [0.000 0.148 0.297 0.394 0.540 0.596 0.743 0.874 0.972 1.116]


In [127]:
print(f"The true Eigenvalues are: {np.array2string(eigvals[:10], formatter={'float': lambda x: f'{x:.3f}'})}")
print(f"The pred Eigenvalues are: {np.array2string(np.array(eigenvalues), formatter={'float': lambda x: f'{x:.3f}'})}")

The true Eigenvalues are: [0.000 0.160 0.425 0.438 0.538 0.612 0.896 1.274 1.496 1.643]
The pred Eigenvalues are: [0.000 0.321 0.581 0.786 0.850 0.902 1.235 1.333 1.361 1.508]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import scipy.sparse as sparse
from tqdm import trange
import copy

# ============ DEVICE SETUP ============
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ============ NETWORK ARCHITECTURE ============
# **CHANGE 1: Increased Default Hidden Dimension to 128**
class Sin(nn.Module):
    """Sine activation function"""
    def forward(self, x):
        return torch.sin(x)


class EigenfunctionNN(nn.Module):
    """
    Neural network to learn eigenfunctions on point clouds.
    Input: 3D coordinates (x, y, z)
    Output: eigenfunction value u(x,y,z) and eigenvalue λ
    """
    def __init__(self, hidden_dim=128, input_dim=3, initial_eigenvalue=0.0):
        super().__init__()
        self.activation = Sin()
        
        # Learnable eigenvalue with better initialization
        self.eigenvalue_layer = nn.Linear(1, 1, bias=False)
        with torch.no_grad():
            self.eigenvalue_layer.weight.fill_(initial_eigenvalue)
        
        # Network layers - ONLY concatenate eigenvalue at the first layer
        self.fc1 = nn.Linear(input_dim + 1, hidden_dim)  # +1 for eigenvalue
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        """
        Args:
            x: (N, 3) point cloud coordinates
        Returns:
            u: (N, 1) eigenfunction values
            eigenvalue: scalar learnable eigenvalue
        """
        # Use torch.clamp(..., min=0.0) to enforce non-negativity
        eigenvalue = torch.clamp(self.eigenvalue_layer(torch.ones(1, 1).to(x.device)), min=0.0)
        eigenvalue_expanded = eigenvalue.expand(x.shape[0], 1)  # (N, 1)
        
        # Forward pass - concatenate eigenvalue ONLY at the input
        h = torch.cat([x, eigenvalue_expanded], dim=1)  # (N, input_dim+1)
        h = self.activation(self.fc1(h))
        
        # No eigenvalue concatenation in hidden layers
        h = self.activation(self.fc2(h))
        h = self.activation(self.fc3(h))
        
        u = self.fc4(h)
        
        return u, eigenvalue


# ============ LOSS COMPUTATION ============
def compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch):
    """
    Compute residual for Lu = λMu using discrete operators.
    Returns:
        loss: MSE of residual ||Lu - λMu||²
    """
    u_flat = u.squeeze()  # (N,)
    
    # Compute Lu and λMu
    Lu = torch.sparse.mm(L_torch, u_flat.unsqueeze(1)).squeeze()
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    residual = Lu - eigenvalue * Mu
    
    return torch.mean(residual ** 2), Lu, Mu


def compute_normalization_loss(u, M_torch):
    """
    Enforce u^T M u = 1 (mass-matrix normalization).
    """
    u_flat = u.squeeze()    
    Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
    norm_squared = torch.dot(u_flat, Mu)

    return (norm_squared - 1.0) ** 2


def compute_orthogonality_loss(u, previous_eigenfunctions, M_torch):
    """
    Enforce u ⊥ u_i for all previously found eigenfunctions.
    Uses M-orthogonality: u^T M u_i = 0
    """
    if not previous_eigenfunctions:
        return torch.tensor(0.0, device=M_torch.device)
    
    u_flat = u.squeeze()
    ortho_loss = torch.tensor(0.0, device=M_torch.device)
    
    
    for u_prev in previous_eigenfunctions:
        u_prev_flat = u_prev.squeeze()
        # Compute u^T M u_prev
        Mu_prev = torch.sparse.mm(M_torch, u_prev_flat.unsqueeze(1)).squeeze()
        overlap = torch.dot(u_flat, Mu_prev)
        ortho_loss += overlap ** 2
    
    return ortho_loss


# **CHANGE 2: Reduced Margin in Ordering Loss**
def compute_ordering_loss(eigenvalue, previous_eigenvalue, margin=1e-4):
    """
    Enforce λ_i > λ_{i-1} + margin.
    """
    if previous_eigenvalue is None:
        return torch.tensor(0.0, device=eigenvalue.device)
    
    # max(0, λ_{i-1} + margin - λ_i)^2
    loss = torch.max(
        torch.tensor(0.0, device=eigenvalue.device), 
        previous_eigenvalue.detach() + margin - eigenvalue
    ) ** 2
    return loss.squeeze()


# ============ UTILITY FUNCTIONS ============
def sparse_to_torch(sparse_matrix, device):
    """Convert scipy sparse matrix to torch sparse tensor."""
    # Handle any scipy sparse format
    if sparse.issparse(sparse_matrix):
        coo = sparse_matrix.tocoo()
    else:
        raise ValueError(f"Expected scipy sparse matrix, got {type(sparse_matrix)}")
    
    indices = torch.LongTensor(np.vstack((coo.row, coo.col)))
    values = torch.FloatTensor(coo.data)
    shape = coo.shape
    return torch.sparse_coo_tensor(indices, values, shape).to(device)


def initialize_weights(m):
    """Reinitialize network weights for finding next eigenfunction."""
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


# ============ MAIN TRAINING FUNCTION ============
def train_eigenvalue_pinn(X, L, M, hidden_dim=128, epochs=20000, 
                          lr=1e-3, num_eigenfunctions=10, 
                          convergence_threshold=1e-7,
                          ortho_weight=100.0,  
                          order_weight=10.0,   # **CHANGE 3: Reduced Order Weight from 100.0 to 10.0**
                          initial_lambda_step=0.2): # **CHANGE 4: Reduced Initial Lambda Step from 0.3 to 0.2**
    """
    Train PINN to solve Lu = λMu eigenvalue problem.
    """
    
    # Prepare inputs
    X_torch = torch.as_tensor(X, dtype=torch.float32, device=device)
    X_torch.requires_grad = True

    # Pre-convert sparse matrices
    L_torch = sparse_to_torch(L, device) if sparse.issparse(L) else L.to(device)
    M_torch = sparse_to_torch(M, device) if sparse.issparse(M) else M.to(device)
    
    # Storage for results
    eigenvalues = []
    eigenfunctions = []
    all_models = []
    loss_history = {'total': [], 'eig': [], 'norm': [], 'ortho': [], 'order': []}
    
    print(f"Training on device: {device}")
    print(f"Point cloud size: {X.shape[0]} points")
    
    # ============ ITERATIVE EIGENFUNCTION DISCOVERY ============
    for eig_idx in range(num_eigenfunctions):
        print(f"\n{'='*60}")
        print(f"Finding eigenfunction {eig_idx + 1}/{num_eigenfunctions}")
        print(f"Hyperparams: λ_step={initial_lambda_step:.2f}, hidden_dim={hidden_dim}, ortho_w={ortho_weight:.1f}, order_w={order_weight:.1f}")
        print(f"{'='*60}")
        
        # Determine previous eigenvalue and initial guess
        previous_eigenvalue = None
        if eig_idx == 0:
            initial_eigenvalue = 0.0
        else:
            previous_eigenvalue = torch.tensor(eigenvalues[-1], dtype=torch.float32, device=device)
            # Use the specified step size
            initial_eigenvalue = eigenvalues[-1] + initial_lambda_step 
            
        
        # Initialize network
        model = EigenfunctionNN(hidden_dim=hidden_dim, input_dim=X.shape[1], 
                               initial_eigenvalue=initial_eigenvalue).to(device)
        
        # Reinitialize weights and biases
        model.apply(initialize_weights) 
        
        # Re-set initial eigenvalue guess, as initialize_weights overwrites it
        if previous_eigenvalue is not None:
             with torch.no_grad():
                model.eigenvalue_layer.weight.fill_(initial_eigenvalue)


        optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.999, 0.9999))
        
        # --- Convergence Tracking Setup ---
        best_model = None
        best_eig_loss = float('inf')
        ema_slope = 1.0
        prev_loss = None
        
        # --- Training Loop ---
        for epoch in trange(epochs, desc=f"Eigen {eig_idx+1}/{num_eigenfunctions}"):
            optimizer.zero_grad()
            
            # Forward pass
            u, eigenvalue = model(X_torch)
            
            # Compute losses
            eig_loss, _, _ = compute_eigenvalue_loss(u, eigenvalue, L_torch, M_torch)
            norm_loss = compute_normalization_loss(u, M_torch)
            ortho_loss = compute_orthogonality_loss(u, eigenfunctions, M_torch)
            order_loss = compute_ordering_loss(eigenvalue, previous_eigenvalue)
            
            # Total loss with weighting
            total_loss = eig_loss + norm_loss + \
                         ortho_weight * ortho_loss + \
                         order_weight * order_loss
            
            # Backward pass
            total_loss.backward()
            optimizer.step()
            
            # Convergence tracking (Use Eig loss for convergence)
            if prev_loss is not None:
                ema_slope = 0.75 * ema_slope + 0.25 * abs(prev_loss - eig_loss.item())
            prev_loss = eig_loss.item()

            # **CHANGE 5: Start convergence check later to ensure stability**
            if ema_slope < convergence_threshold * 10.0 and epoch > 8000: 
                print(f"Converged at epoch {epoch}")
                break

            # Save best model based on Eig loss
            if eig_loss.item() < best_eig_loss:
                best_eig_loss = eig_loss.item()
                best_model = copy.deepcopy(model)

            # Logging every 500 epochs
            if epoch % 500 == 0:
                print(f"Epoch {epoch:5d} | λ={eigenvalue.item():.6f} | "
                      f"Eig={eig_loss.item():.2e} | Norm={norm_loss.item():.2e} | "
                      f"Ortho={ortho_loss.item():.2e} | Order={order_loss.item():.2e} | "
                      f"EMA slope={ema_slope:.2e}")

            # Lightweight history
            if epoch % 100 == 0:
                loss_history['total'].append(total_loss.item())
                loss_history['eig'].append(eig_loss.item())
                loss_history['norm'].append(norm_loss.item())
                loss_history['ortho'].append(ortho_loss.item())
                loss_history['order'].append(order_loss.item())

        # Store results
        if best_model is None:
             print("Warning: No best model found (first epoch was best, using final model).")
             best_model = model
             
        with torch.no_grad():
            u_final, eigenvalue_final = best_model(X_torch)
            eigenvalues.append(eigenvalue_final.item())
            
            # Re-normalize the eigenfunction after training (u^T M u = 1)
            u_flat = u_final.squeeze()
            Mu = torch.sparse.mm(M_torch, u_flat.unsqueeze(1)).squeeze()
            norm_squared = torch.dot(u_flat, Mu)
            u_normalized = u_final / torch.sqrt(norm_squared)

            eigenfunctions.append(u_normalized.detach())
            all_models.append(best_model)

        print(f"\nFound eigenvalue: λ_{eig_idx} = {eigenvalue_final.item():.6f}")
    
    return eigenvalues, eigenfunctions, all_models, loss_history